# One–Level Additive Schwarz Method: Theory and FEM Implementation

### 1.Introduction
   

 We consider the finite element discretization of a second order elliptic boundary value problem and use domain decomposition methods as preconditioners. The additive Schwarz method is used as a preconditioner for Krylov subspace methods. Here, we employ ASM as a preconditioner for the Conjugate Gradient method and examine its impact on convergence. The results illustrate the effectiveness of one–level ASM and limitation in propagating global information, motivating the use of two–level methods.

### Model Problem

Let $\Omega \subset \mathbb{R}^2$ be a open, bounded domain.
We consider the Poisson problem

$$\Delta u = f \quad \text{in } \Omega,$$ 
$$u = g \quad \text{on } \partial\Omega \tag{1}.$$
where g is a given function and this is a Dirichlet problem. When g =0 it is said to be homogneous.


### Weak Formulation
Equation (1) is the "Strong formulation" of the Poisson problem. An alternative formulation is required in order to reduce the order of derivatives imposed on the unknown solution $u$. 
This allows the second order differential problem to become a first order problem in the integral form, called the "weak formulation" of the differential problem. It proceeds by multiplying the strong formulation by an test function $v$ and integrate over the domain $\Omega$:
$$
\int_\Omega -\Delta u \, v \, d\Omega = \int_\Omega f v \, d\Omega.
$$
Analougous of using integration by parts for 1D, we apply Gauss theorem and restricting the test functions to vanish on
$\partial\Omega$, the boundary terms vanish and the weak formulation becomes
$$
\int_\Omega \nabla u \cdot \nabla v \, d\Omega = \int_\Omega f v \, d\Omega.
\tag{2}
$$
In order to make sense of (2), define the Sobolev space
$$
H_0^1(0,1) = \{ v \in H^1(0,1) : v(0) = v(1) = 0 \}
$$
where 
$$H^1(0,1) = \{v \in L^2(0,1) : v' \in L^2(0,1)\}.$$
The problem can now be stated as follows:

**Weak Formulation**

find $u \in V = H_0^1(0,1)$ such that
$$
a(u,v) = \ell(v) \quad \forall v \in V,
\tag{3}
$$
where
$$
a(u,v) = \int_\Omega \nabla u \cdot \nabla v \, dx,
\qquad
\ell(v) = \int_\Omega f v \, dx.
$$

The existence and uniqueness of the weak formulation is ensured by the Lax-Milgram lemma.

**Lax-Milgram Lemma**
Let $V$ be a Hilbert space, let $a(\cdot,\cdot) : V \times V \to \mathbb{R}$
be a continuous and coercive bilinear form, and let
$F(\cdot) : V \to \mathbb{R}$ be a linear and continuous functional.
Then there exists a unique solution $u \in V$ such that
$$
a(u,v) = F(v) \quad \forall v \in V.
$$


### Finite Dimensional Galerkin Approximation
To obtain a computable approximation of the problem, 
we replace $V$ by a finite-dimensional subspace $V_h \subset V$.
The finite dimensional problem is as follows 

Find $u_h \in V_h$ such that
$$
a(u_h, v_h) = F(v_h) \quad \forall v_h \in V_h.
\tag{4}
$$
Since $V_h$ is finite-dimensional, this problem reduces to the solution
of a linear system of algebraic equations.   

### Finite Element Discretization

Let $\Omega \subset \mathbb{R}^2$ be a bounded polygonal domain.
We consider a triangulation $\mathcal{T}_h$ of $\Omega$ into non-overlapping
triangular elements, where $h$ denotes the mesh size.

Associated with the triangulation, we define the finite element space
$V_h \subset H_0^1(\Omega)$ as
$$
V_h = \{ v_h \in H_0^1(\Omega) : v_h|_K \in \mathbb{P}_r(K)
\ \forall K \in \mathcal{T}_h \},
$$
where $\mathbb{P}_r(K)$ denotes the space of polynomials with degree lower than or equal to r in the variable x on the element $K$.

Since $V_h$ is finite-dimensional space it admits a basis
$\{ \varphi_i \}_{i=1}^N$, where each basis function is associated with a
vertex of the triangulation $\mathcal{T}_h$. These basis functions are
referred to as nodal basis functions.

Each basis function $\varphi_i$ satisfies
$$
\varphi_i(x_j) = \delta_{ij},
$$
where $\{x_j\}$ denotes the set of mesh vertices and $\delta_{ij}$ is the
Kronecker delta. Moreover, $\varphi_i$ is piecewise continuous on each element
and has local support, being nonzero only on the elements sharing the
vertex $x_i$.

Using this basis, the finite element solution is expressed as
$$
u_h = \sum_{i=1}^N \alpha_i \varphi_i,
$$
where $\{\alpha_i\}$ are the coefficients of the solution in the chosen basis.


The finite element approximation of the weak problem is defined as follows:

Find $u_h \in V_h$ such that
$$
a(u_h, v_h) = F(v_h) \quad \forall v_h \in V_h.
$$

Substituting the expansion of $u_h$ into the (4) yields
a linear system of algebraic equations
$$
A u = f,
\tag{5}
$$
where the stiffness matrix $A \in \mathbb{R}^{N \times N}$ and the load
vector $f \in \mathbb{R}^N$ are given by
$$
A_{ij} = a(\varphi_j, \varphi_i), \qquad
f_i = F(\varphi_i).
$$


**NGSolve Python** interface allows the mathematical equation to be expressed in its computational form.

In [21]:
from ngsolve import *
from ngsolve.webgui import Draw

The unit square $ \Omega = (0,1)^2 $ triangulation or mesh with mesh size h = 0.2.

In [22]:
mesh = Mesh(unit_square.GenerateMesh(maxh = 0.2))
Draw (mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

BaseWebGuiScene

fes stands for Finite Element Space and here we have created a quadratic, continuous finite element space on the given mesh, enforcing zero boundary conditions everywhere, and fes.ndof tells the number of basis functions.

In [23]:
# Define the Finite Element Space
fes = H1(mesh, order=2, dirichlet='bottom|left|top|right')
print ("number of dofs =", fes.ndof)
print ("mesh = ", mesh.nv)

number of dofs = 129
mesh =  38


In [24]:
# Define Bilinear form
u = fes.TrialFunction()
v = fes.TestFunction()
a = BilinearForm(grad(u)*grad(v)*dx)

In [25]:
# RHS
funct = x**2 * (1-y)**2
f = LinearForm(funct*v*dx)

In [26]:
#Matrix Computation
a.Assemble()
f.Assemble()

GridFunction stores the FEM solution, the matrix inverse solves the linear system on free DOFs, and Draw(gfu) visualizes the resulting finite element approximation $u_h$

In [27]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec #u = inv(A).f
Draw (gfu);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

### Iterative Solver
Equation (5) has been explicitly solved by applying the inverse of the stiffness matrix, as illustrated by the code line. This approach becomes expensive as the number of degrees of freedom increases. In such settings, iterative solvers such as the CG method, GMRES method and etc. can be used.
However the efficiency of the iterative solvers depends on the conditioning of the linear system. As a result, the use of iterative solvers naturally leads to the preconditioners.

### Schwarz Iterative methods
The Schwarz method in its original form was proposed by H. Schwarz as an
iterative scheme to prove the existence of solutions to elliptic partial
differential equations. In modern numerical analysis, however Schwarz
methods are primarily used as **domain decomposition preconditioners** for iterative solvers such as the Conjugate
Gradient method and more generally, Krylov subspace methods.

The Schwarz methods are based on an overlapping
decomposition of the computational domain. Let $\Omega \subset \mathbb
{R}^d$ be the computational domain and let $\tau_h$ be a finite element triangulation of $\Omega$. The domain is decomposed into a collection of overlapping subdomains
$$
\Omega = \{ \Omega_i \}_{i=1}^N,
\quad {\Omega_i} \cap {\Omega_{j}} \neq \emptyset, \quad i \neq j $$
where each subdomain covers a portion of the global domain and overlaps with
its neighbors. 

Here we decompose the domain $\Omega $ into a collection of non-overlapping subdomains. In the finite element setting, overlap is not imposed geometrically but is instead induced by shared degrees of freedom across neighboring subdomains.

In [28]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
mx, my = 3,3
rects = [MoveTo(i/mx,j/mx).Rectangle(1/mx,1/my).Face().bc("mat"+str(i)+str(j)) for i in range(mx) for j in range(my)]
shape = Glue(rects)
Draw(shape)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

BaseWebGuiScene

In NGSolve, subdomains are represented via material labels. These labels allow us to associate degrees of freedom with each subdomain.

In [29]:
geo = OCCGeometry(shape,dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
print ("Subdomains = ", mesh.GetMaterials())
Draw (mesh);

Subdomains =  ('mat00', 'mat01', 'mat02', 'mat10', 'mat11', 'mat12', 'mat20', 'mat21', 'mat22')


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

### Algebraic form of Schwarz method
The finite element discretization of the model problem leads to a linear system
of algebraic equations of the form
$$
A u = f,
$$
where$A \in \mathbb{R}^{N_h \times N_h}$is the stiffness matrix associated with the bilinear form,$u \in \mathbb{R}^{N_h}$is the vector of unknown coefficients, and $f \in \mathbb{R}^{N_h}$ is the load vector. Here $N_h$ denotes the total number of interior degrees of freedom of the triangulation.

We consider a decomposition of the domain $\Omega$ into two overlapping subdomains $ \Omega_1 \quad \text{and} \quad \Omega_2.$Let $N_1 \quad \text{and} \quad N_2$ denote the number of degrees of freedom associated with $\Omega_1 \quad \text{and} \quad \Omega_2,$ respectively. Due to the overlap, one generally has $$N_h \le N_1 + N_2.$$

In order to work with vectors defined on the global domain, we introduce
restriction operators
$$
R_i : \mathbb{R}^{N_h} \rightarrow \mathbb{R}^{N_i}, \quad i = 1,2,
$$
which extract the components of a global vector corresponding to the degrees of
freedom associated with the subdomain$\Omega_i.$ The corresponding extension (or prolongation) operators are given by the
transposes $R_i^T,$ which extend a local vector by zero to the full domain.

Using these operators, the local stiffness matrices are defined as
$$
A_i = R_i A R_i^T \in \mathbb{R}^{N_i \times N_i}, \quad i = 1,2.
$$
These matrices correspond to finite element problems posed on the subdomains
$
\Omega_i
$
with homogeneous Dirichlet boundary conditions imposed on the artificial
interfaces.


### Additive Schwarz Iterative Method

We define the operators
$$
Q_i = R_i^T A_i^{-1} R_i, \quad i = 1,2.
$$

An iteration of the additive Schwarz method applied to the linear system
$
A u = f
$
is given by
$$
u^{(k+1)} = u^{(k)} + (Q_1 + Q_2)\bigl(f - A u^{(k)}\bigr).
$$

This iteration can be interpreted as follows. At each iteration step, the
residual
$
f - A u^{(k)}
$
is first restricted to each subdomain. On each subdomain, an independent local
problem involving the local stiffness matrix is solved. The resulting local
corrections are then extended by zero to the global domain and summed to obtain
the update of the global solution.

The additive Schwarz iteration naturally extends to a decomposition of the
domain
$
\Omega
$
into
$
M \ge 2
$
overlapping subdomains
$
\{\Omega_i\}_{i=1}^M.
$
In this case, the iteration takes the form
$$
u^{(k+1)} = u^{(k)} + \left( \sum_{i=1}^M Q_i \right)\bigl(f - A u^{(k)}\bigr).
$$


### Schwarz Preconditioners – One-Level Method

The above iteration can be rewritten in the standard form of a preconditioned fixed-point iteration. In particular, it corresponds to a Richardson iteration preconditioned by the additive Schwarz operator.

Introducing the operator
$$
P_{as}^{-1} = \sum_{i=1}^{M} Q_i,
$$
the additive Schwarz iteration can be written as
$$
u^{(k+1)} = u^{(k)} + P_{as}^{-1} \left( f - A u^{(k)} \right).
$$

This shows that the additive Schwarz method corresponds to Richardson iterations applied to the linear system
$$
A u = f,
$$
preconditioned by the operator $P_{as}$. For this reason, $P_{as}$ is referred to as the **additive Schwarz preconditioner**.

**Remark:** 

An important special case arises when the subdomains $\Omega_i$ are disjoint, that is, when no overlap is present. In this situation, the additive Schwarz preconditioner reduces to the classical block Jacobi preconditioner.

If the global stiffness matrix $A$ is reordered according to the subdomain partition, the block Jacobi preconditioner retains only the diagonal blocks:
$$
P_J =
\begin{pmatrix}
A_1 & 0 & \cdots & 0 \\
0 & A_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & A_M
\end{pmatrix},
\qquad
P_J^{-1} =
\begin{pmatrix}
A_1^{-1} & 0 & \cdots & 0 \\
0 & A_2^{-1} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & A_M^{-1}
\end{pmatrix}.
$$

The additive Schwarz preconditioner can therefore be viewed as a natural generalization of block Jacobi, where overlap between subdomains is introduced in order to improve convergence.

Using the definition of the operators
$$
P_i = R_i^{T} A_i^{-1} R_i A,
$$
the preconditioned matrix associated with the additive Schwarz method can be written as
$$
Q_a = P_{as}^{-1} A = \sum_{i=1}^{M} P_i.
$$

**Results**

A key theoretical result is that each operator $P_i$ is symmetric and non-negative with respect to the scalar product induced by $A$, defined as
$$
(w, v)_A = (A w, v).
$$

As a consequence, the operator $Q_a$ is symmetric and positive definite with respect to the $A$-induced inner product. This property is crucial, as it guarantees that Krylov methods such as the **preconditioned conjugate gradient method** can be safely applied.

### Construction of Overlapping Subdomains

The construction of overlapping subdomains typically proceeds in two stages.

First, the domain $\Omega$ is partitioned into a set of $M$ disjoint subdomains $\{ \hat{\Omega}_i \}_{i=1}^{M}$. This initial partition can be obtained either by geometric decomposition or by graph partitioning techniques.

Given a non-overlapping subdomain, its extension by one layer of
finite elements is defined as

$$
{\Omega}_j
=
\mathrm{int}
\left(
\bigcup_{k \in \mathrm{dof}(\Omega_j)}
\mathrm{supp}(\varphi_k)
\right).
$$

where $\{\varphi_k\}$ denotes the finite element basis functions and
$\operatorname{dof}(\Omega_j)$ is the set of degrees of freedom associated
with $\Omega_j$.

In a second step, each disjoint subdomain $\hat{\Omega}_i$ is extended by adding neighboring layers of finite elements within a distance $\delta$. The resulting extended regions define the overlapping subdomains $\Omega_i$.

The overlap parameter $\delta$ is often chosen proportional to the mesh size $h$. The case $\delta = h$ corresponds to **minimum overlap**, where the overlap consists of a single layer of elements.

**definition:** :Local subspaces

For each overlapping subdomain $\Omega_j$, define the subspace

$$ V_{h,0}(\Omega_j)
={ v_h \in V_h \;:\; \operatorname{supp}(v_h) \subset \Omega_j}.$$

Functions in $V_{h,0}(\Omega_j)$ vanish outside $\Omega_j$.

**lemma:** Partition of unity property

For every $v_h \in V_h$, there exist functions
$
v_j \in V_{h,0}(\Omega_j), \quad j = 1, \ldots, N,
$
such that
$
v_h = \sum_{j=1}^N v_j .
$




We solve the 2D Poisson problem on the unit square using the Finite Element Method. 
We compare two approaches for solving the resulting linear system $A u = f$:

1. **Standard Conjugate Gradient (CG)** without preconditioning.
2. **Conjugate Gradient with Additive Schwarz Method (ASM) preconditioner**.

This allows us to observe the effect of the ASM preconditioner on the condition number and the number of iterations.


In [30]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *    #OCC geometry tools

We create a $m_x \times m_y$ disjoint subdomain and each has its own material label (subij)

In [31]:
mx, my = 3,3           
rects = [MoveTo(i/mx,j/mx).Rectangle(1/mx,1/my).Face().bc("sub"+str(i)+str(j)) 
         for i in range(mx) for j in range(my)]
shape = Glue(rects)
shape.edges[Y<0.001].name = "bottom"
shape.edges[Y>1-0.001].name = "top"
shape.edges[X<0.001].name = "left"
shape.edges[X>1-0.001].name = "right"

Draw (shape);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [32]:
geo = OCCGeometry(shape,dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
print (mesh.GetMaterials())
Draw (mesh);

('sub00', 'sub01', 'sub02', 'sub10', 'sub11', 'sub12', 'sub20', 'sub21', 'sub22')


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

In [33]:
fes = H1(mesh, order=2, dirichlet='bottom|left|top|right')
print ("ndof =", fes.ndof)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()

ndof = 453


#### Construction of Additive Schwarz preconditioner

{fes.GetDofs(dom)} - degrees of freedom supported in $\Omega_i$
This corresponds to the restriction operator $R_i$

$$R_i : V_h \longrightarrow V_h(\Omega_i)$$


fes.GetDofs(dom) } $\cap$ FreeDofs() - enforcement of homogeneous Dirichlet boundary conditions

$$A_i = R_i \, A \, R_i^{T}$$

$
A_i^{-1}$  represents the local subdomain solve on $\Omega_i$

One-level additive Schwarz preconditioner:  

$$ P_{AS}^{-1} = \sum_{i=1}^{M} R_i^{T} A_i^{-1} R_i $$ 


In [34]:
pre = None
for dom in mesh.Materials(".*").Split():
    domaindofs = fes.GetDofs(dom) & fes.FreeDofs()
    print ("num dofs:", domaindofs.NumSet())
    invi = a.mat.Inverse(domaindofs)
    pre = invi if pre == None else pre + invi

num dofs: 48
num dofs: 54
num dofs: 48
num dofs: 54
num dofs: 57
num dofs: 50
num dofs: 48
num dofs: 50
num dofs: 44


In [35]:
from ngsolve.la import EigenValues_Preconditioner
lami = list(EigenValues_Preconditioner(mat=a.mat, pre=pre))
print ("condition number of the preconditioned matrix = ", lami[-1]/lami[0])

condition number of the preconditioned matrix =  13.906733524542254


In [36]:
from ngsolve.krylovspace import CGSolver
from ngsolve import GridFunction

funct = x**2 * (1-y)**2
f = LinearForm(funct*v*dx).Assemble()
gfu = GridFunction(fes)

inv = CGSolver(mat=a.mat, pre=pre, tol=1e-8, maxiter=500, printrates=True)
gfu.vec.data = inv * f.vec
print ("Iterations with ASM: ", len(inv.errors))
Draw (gfu)


CG iteration 1, residual = 0.017532841870695044     
CG iteration 2, residual = 0.013173327221003938     
CG iteration 3, residual = 0.009280248795955907     
CG iteration 4, residual = 0.0052043123778143735     
CG iteration 5, residual = 0.002081166547290058     
CG iteration 6, residual = 0.0013048259323649978     
CG iteration 7, residual = 0.0004812625154856358     
CG iteration 8, residual = 0.0001607234282132628     
CG iteration 9, residual = 6.153243017777783e-05     
CG iteration 10, residual = 2.2622466624488308e-05     
CG iteration 11, residual = 6.588380086422677e-06     
CG iteration 12, residual = 1.9890527196386606e-06     
CG iteration 13, residual = 7.391706366910892e-07     
CG iteration 14, residual = 2.0711903312281166e-07     
CG iteration 15, residual = 8.109483028454211e-08     
CG iteration 16, residual = 2.2947611185358595e-08     
CG iteration 17, residual = 9.51911738587102e-09     
CG iteration 18, residual = 3.2084495015167807e-09     
CG iteration 19, re

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

BaseWebGuiScene

### Visualizing Ri

In [37]:
gfi = GridFunction(fes)
domi = mesh.Materials(".*").Split()[7]
proj = Projector(fes.GetDofs(domi), True)
gfi.vec.data = proj * gfu.vec
Draw (gfi);


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

In [38]:
#Solve without Preconditioner 
gfu_nopre = GridFunction(fes)
cg_nopre = CGSolver(mat=a.mat,freedofs=fes.FreeDofs(),tol=1e-8,maxiter=500, printrates=True)

gfu_nopre.vec.data = cg_nopre * f.vec
print("Iterations (no preconditioner):", len(cg_nopre.errors))

Draw(gfu_nopre)

CG iteration 1, residual = 0.015876178507109585     
CG iteration 2, residual = 0.01580356940642441     
CG iteration 3, residual = 0.012943055400778038     
CG iteration 4, residual = 0.008747655324898093     
CG iteration 5, residual = 0.006757705554658536     
CG iteration 6, residual = 0.0056588521438040735     
CG iteration 7, residual = 0.004468036501119036     
CG iteration 8, residual = 0.0033209128845980004     
CG iteration 9, residual = 0.0025135696793777834     
CG iteration 10, residual = 0.0017977782357875155     
CG iteration 11, residual = 0.0013140654122018307     
CG iteration 12, residual = 0.001062803981639732     
CG iteration 13, residual = 0.001186912223064372     
CG iteration 14, residual = 0.001344291559482298     
CG iteration 15, residual = 0.001332567308954171     
CG iteration 16, residual = 0.001106887747301054     
CG iteration 17, residual = 0.0006634508371432657     
CG iteration 18, residual = 0.0004562848624917734     
CG iteration 19, residual = 0.0

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.26…

BaseWebGuiScene

In [39]:
from ngsolve.la import EigenValues_Preconditioner
I = IdentityMatrix(fes.ndof, complex=False)
lami_nopre = list(EigenValues_Preconditioner(mat=a.mat, pre =I))
print ("condition number of the unpreconditioned matrix = ", lami_nopre[-1]/lami_nopre[0])

condition number of the unpreconditioned matrix =  29854.258380865114


#### Observations
The performance of the conjugate gradient method was studied with and without the one-level additive Schwarz (ASM) preconditioner.
The numerical results confirm that the one-level additive Schwarz preconditioner converged in 21 iterations to reach the prescribed tolerance of $10^{-8}$ and the residual decreased rapidly.
In contrast, the unpreconditioned CG method required 98 iterations to reach the same tolerance and residual history shows significantly slower decay.